In [3]:
import os
import pandas as pd
from netaddr import *
import socket


class Firewall:
   
    def __init__(self,rulesFilePath):
        self.rulesFilePath = rulesFilePath
        df = pd.DataFrame(columns=["direction","protocol","port","ip_address"])    
        if(os.path.isfile(self.rulesFilePath)):
            df = pd.read_csv(self.rulesFilePath)
           
            df["isRange"] =  df.ip_address.apply(lambda x : True if "-" in x else False)
            df["startIpRange"] = df.apply(lambda x : int(IPAddress(x["ip_address"].split("-")[0])) if x["isRange"] else 0,axis=1)
            df["endIpRange"] = df.apply(lambda x : int(IPAddress(x["ip_address"].split("-")[-1])) if x["isRange"] else 0,axis=1)
            df["isPortRange"] =  df.port.apply(lambda x : True if "-" in x else False)
            df["startPortRange"] = df.apply(lambda x : int(x["port"].split("-")[0]) if x["isPortRange"] else 0,axis=1)
            df["endPortRange"] = df.apply(lambda x : int(x["port"].split("-")[-1]) if x["isPortRange"] else 0,axis=1)
        else:
            df = pd.DataFrame(columns=["direction","protocol","port","ip_address"])    
        self.df = df
           
   
    def accept_packet(self,direction=None,protocol=None,port=None,ip_address=None):
       
        if(len(self.df)==0):
            return False
        df = self.df
        result = df[(df['direction']==direction) &
                         (df['protocol']==protocol) &
                         (df['port']==port) &
                         (df['ip_address']==ip_address) ]
       
        intIPAddress  = int(IPAddress(ip_address))
        intPortValue = int(port)
        if(len(result)>0):
            return True
       
        result = df[(df['direction']==direction) &
                         (df['protocol']==protocol) &
                         (df['port']==port) &
                         (df['isRange']==True) &
                         (
                             (df['startIpRange']<=intIPAddress) &
                             (df['endIpRange']>=intIPAddress)
                         )]
   
        if(len(result)>0):
            return True
       
       
        result = df[(df['direction']==direction) &
                         (df['protocol']==protocol) &
                         (df['ip_address']==ip_address) &
                         (df['isPortRange']==True) &
                         (
                             (df['startPortRange']<=intPortValue) &
                             (df['endPortRange']>=intPortValue)
                         )]
       
        if(len(result)>0):
            return True
       
       
        result = df[(df['direction']==direction) &
                         (df['protocol']==protocol) &
                         (df['isRange']==True) &
                         (df['isPortRange']==True) &
                         (
                             (df['startIpRange']<=intIPAddress) &
                             (df['endIpRange']>=intIPAddress)
                         ) &
                         (
                             (df['startPortRange']<=intPortValue) &
                             (df['endPortRange']>=intPortValue)
                         )
                        ]
        if(len(result)>0):
            return True
       
        else:
            return False
       
       
       
   



fireWallRules = Firewall("./input.csv")

#Test case showing Accuracy of packets being predicted
resultsDB = 0
datasetCount = 0
for line in open("testDataset.csv","r").readlines():
    line = line.strip().split(",")
    if(str(fireWallRules.accept_packet(line[0],line[1],line[2],line[3])) == line[4]):
        resultsDB= resultsDB + 1
    datasetCount = datasetCount + 1
    break
print("Accuracy : {}".format(round(resultsDB/datasetCount*100,2)))
   

str(fireWallRules.accept_packet(line[0],line[1],line[2],line[3])) 


Accuracy : 100.0


'True'

In [7]:
#Test
fireWallRules.accept_packet("outbound","udp","1000","10.10.10.0")

True